# **Realeboha Raymond Ramphielo 10622234**

  **B9AI108 Programming For Data Analysis B9AI108_2223_TMD1S**
  **CA 2**

  [link text](https://colab.research.google.com/drive/1XIZa4PnihdmvbuFt9s89LH5AF3aYZIFr?usp=sharing)

## **Intoduction**
The plan is to scrap the Indeed.com website *https://ie.indeed.com/?from=gnav-homepage*. My plan is to gather information on a general driving job vacany, how long the ad has been posted and pay scale.

*   irishjobs.ie -> problems with the requested data, cant locate it
*   indeed.com -> https response 403
*   datasciencejobs -> salary, location display funny on inspect, svg
*   pnet -> does not have salary
*   https://www.careerjunction.co.za -> might just work






## Libarary intallations

In [27]:
pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 127 kB 5.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.13
    Uninstalling urllib3-1.26.13:
      Successfully uninstalled urllib3-1.26.13
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
selenium 4.6.1 requires urllib3[socks]~=1.26, but you have urllib3 1.25.11 which is incompatible.


In [28]:
pip install bs4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [29]:
pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [30]:
pip install strings

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached strings-0.1.2.tar.gz (2.2 kB)
  Using cached strings-0.1.1.tar.gz (2.2 kB)
ERROR: Could not find a version that satisfies the requirement strings (from versions: 0.1.1, 0.1.2)
ERROR: No matching distribution found for strings


In [31]:
pip install selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached urllib3-1.26.13-py2.py3-none-any.whl (140 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.13 which is incompatible.


In [32]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from datetime import datetime, date, timedelta 
import string

## **Steps**

In [ ]:
#default leocation = 0
locations = {'Gauteng':'2747', 'Western Cape':'16149', 'KwaZulu-Natal':'13131',\
             'Eastern Cape':'2', 'Free State':'1782', 'Mpumalanga':'14867',\
             'Limpopo':'14594', 'North West':'15372', 'Northern Cape':'15837',\
             'International':'1000001', 'Working From Home':'100000'} 

In [108]:
def get_url(job, location = 0):
  
  '''This function sets up the search url from the user defines job and location,
    default location will be 0'''

  #When no location is specified, default location = 0, i.e. the serch will not
  # have any location restrictions

  locations = {'Gauteng':'2747', 'Western Cape':'16149', 'KwaZulu-Natal':'13131',\
              'Eastern Cape':'2', 'Free State':'1782', 'Mpumalanga':'14867',\
              'Limpopo':'14594', 'North West':'15372', 'Northern Cape':'15837',\
              'International':'1000001', 'Working From Home':'100000'}

  #above is the locations dictionary that will encode the location from a normal
  #word search to an 'int' search
  
  error_message = 'No access to location, please revise'
  
  try:
    if location != 0:                    #encode the location
      location = locations[location]
    gen_url = "https://www.careerjunction.co.za/jobs/results?keywords={}&autosuggestEndpoint=%2Fautosuggest&location={}&category=&btnSubmit=+"
    search_url = gen_url.format(job, location)
    
    return search_url
  
  except KeyError:
    return error_message

In [109]:
def make_soup(job, location=0):
  
  '''The function creates a response from the created by get_url, then scraps 
    the actual data from the website'''

  gen_url = get_url(job, location)    #defines the website that will be scraped
  job_collection = []    #empty list to contain all the extracted job attributes 

  for i in range(1,5):
    search_page = f"&page={str(i)}"          #this is the start page
    url = gen_url + search_page             #add a page to the url
    response = requests.get(url)  

    pasta = BeautifulSoup(response.content, 'html.parser') #create the soup
    jobs = pasta.find('div', class_="two-thirds")          #location of the jobs on the page
    j_cards = jobs.find_all('div', class_="job-result")    #collection of the jobs on the page
    
    for j_card in j_cards:  #looping over the job cards list to extract features of each job
      a_job = {}            #dictionary to hold individual job information
      
      prim_link = "https://www.careerjunction.co.za"
      j_today = date.today()
      j_title = j_card.find('h2')
      
      a_job['Job Title'] = j_title.text.strip()
      #company_name = j_card.find('h3')
      if j_card.find('h3') is not None:
        a_job['Company'] = j_card.find('h3').text.strip()
      a_job['Location'] = j_card.find('li', class_="location").text.strip()
      a_job['Job Type'] = j_card.find('li', class_="position").text.strip()
      a_job['Salary'] = j_card.find('li', class_="salary").text.strip('R').strip()
      a_job['Posting Date'] = j_card.find('li', class_="updated-time").text\
      .strip('Posted').strip()

      j_expiry = j_card.find('li', class_="expires").text.split()[2]
      try:
        ini_exp = j_today + timedelta(days=int(j_expiry))
      except ValueError:
        ini_exp = j_today
      a_job['Expiry Date'] = ini_exp.strftime('%d %b %Y')
      a_job['Link'] = prim_link + j_title.find('a').get('href')

      job_collection.append(a_job) 
      #adds the single jobs dictionary to the jobs collection list, feature names are the keys

  return job_collection    #returns jobs list


In [110]:
def main_jobs(job, location=0):

  '''Runs make_soup functions then creates a pandas dataframe that will be
    converted into a csv file'''

  j_data = make_soup(job, location)  #defines a list of dictionaries
  df = pd.DataFrame(j_data)          #coverts the list above to a dataframe
  #df['Monthly Salary'] = df['Salary']
  #df['Monthly Salary'] = df[df['Salary'].str.contain('per week', case=False)]


  df.to_csv('jobs_data.csv')   #creates a jobs_data csv file from the dataframe

  return df 

In [112]:
some_some = main_jobs('data', 'Gauteng')


In [113]:
some_some

,Job Title,Company,Location,Job Type,Salary,Posting Date,Expiry Date,Link
0,Data Modeler,Lusapho Professional Services,"Johannesburg Region, Gauteng\n / Work From Home",Contract Specialist position,Undisclosed,26 Nov 2022,21 Dec 2022,https://www.careerjunction.co.za/data-modeler-...
1,Data Analyst (Power BI),Network Contracting Solutions,"Johannesburg North (Johannesburg North), Gauteng",Contract Senior position,400.00 - R480.00 per hour,25 Nov 2022,07 Dec 2022,https://www.careerjunction.co.za/data-analyst-...
2,People Data Analyst at SA Taxi Development Fin...,SA Taxi Development Finance,"Midrand, Gauteng",Permanent Intermediate EE position,Undisclosed,25 Nov 2022,11 Dec 2022,https://www.careerjunction.co.za/people-data-a...
3,Data Engineer Services,Tipp Focus,"Pretoria CBD, Gauteng",Contract Senior position,Undisclosed,25 Nov 2022,29 Dec 2022,https://www.careerjunction.co.za/data-engineer...
4,Junior Data Manager (12 Months Contract) Wits ...,AJ Personnel,"Johannesburg Region, Gauteng",Contract Junior position,Undisclosed,25 Nov 2022,25 Dec 2022,https://www.careerjunction.co.za/junior-data-m...
...,...,...,...,...,...,...,...,...
95,Business Process Engineer,RAD Resources SA,"Bryanston, Gauteng",Permanent Management position,Undisclosed,25 Nov 2022,29 Dec 2022,https://www.careerjunction.co.za/business-proc...
96,SENIOR SYSTEMS (COMPLIANCE ANALYST),M-Ploy Global Resourcing,"Centurion, Gauteng",Permanent Specialist EE position,Undisclosed,26 Nov 2022,12 Dec 2022,https://www.careerjunction.co.za/senior-system...
97,Invoice Clerk,Manpower SA,"Johannesburg North (Johannesburg North), Gauteng",Permanent Intermediate position,"8,000 - R10,000 per month",25 Nov 2022,15 Dec 2022,https://www.careerjunction.co.za/invoice-clerk...
98,Reporting Analyst,DAV,"Sandown, Gauteng",Permanent Specialist position,"40,000 - R42,000 per month",25 Nov 2022,11 Dec 2022,https://www.careerjunction.co.za/reporting-ana...


In [90]:
p_play = 'R ds dsdR'
p_play2 = p_play.strip('R')
p_play2

' ds dsd'

In [99]:
li = []
for i in range(1,3):
  gen_url3 = "https://www.careerjunction.co.za/jobs/results?keywords=data&autosuggestEndpoint=%2fautosuggest&location=2747&category=&btnSubmit=+"
  #url1 = get_url('data', 'Limpopo')
  cococ = f"&page={str(i)}"
  gen_url = gen_url3 + cococ
  resp = requests.get(gen_url)
  #print(resp, resp.reason)
  #print(gen_url)
  #print(url1)
  #resp.status_code
  pasta = BeautifulSoup(resp.content, 'html.parser')
  jobs = pasta.find('div', class_="two-thirds")
  j_cards = jobs.find_all('div', class_="job-result")
  #j_cards.pop(5)
  print(len(j_cards))
  #j_cards
  #li = []
  for j_card in j_cards:
    j_today = date.today()
    a_job = {}
    titleofjob = j_card.find('h2')
    a_job['Job Title'] = (titleofjob.text.strip())
    a_job['Company'] = (j_card.find('h3').text.strip())
    a_job['Location'] = (j_card.find('li', class_="location").text.strip())
    a_job['Job Type'] = (j_card.find('li', class_="position").text.strip())
    a_job['Salary'] = (j_card.find('li', class_="salary").text.strip('R').strip())
    a_job['Posting date'] = (j_card.find('li', class_="updated-time").text.strip('Posted').strip())
    j_expiry = j_card.find('li', class_="expires").text.split()[2]
    if j_expiry != 'today':
      ini_exp = j_today + timedelta(days=int(j_expiry))
      a_job['Expiry date'] = ini_exp.strftime('%d %b %Y')
    else:
      a_job['Expiry date'] = j_today.strftime('%d %b %Y')
    #a_job['Job URL'] = (j_card.find('h2').text.strip())
    #a_job['j_title'] = (j_card.find('h2').text.strip())
    a_job['Link']  = 'https://www.careerjunction.co.za'+titleofjob.find('a').get('href')

    li.append(a_job)

  #two-way step in accessing an embedded feature, find outer tag and assign it
  #then use asigned_varaible.attrs['some_tag'] to locate the embedded feature
  #eg -> var = j_card.find('h2') followed by feature = var.attrs['a'] 

  #convert final list of dictionaries to a pandas dataframe then use to_csv after
  #df = pd.DataFrame(jobs) followed by df.to_csv('file_name.csv')

  #len(li)
  #li[3]
li


25
25


[{'Job Title': 'Data Modeler',
  'Company': 'Lusapho Professional Services',
  'Location': 'Johannesburg Region, Gauteng\n\xa0/ Work From Home',
  'Job Type': 'Contract Specialist position',
  'Salary': 'Undisclosed',
  'Posting date': '26 Nov 2022',
  'Expiry date': '21 Dec 2022',
  'Link': 'https://www.careerjunction.co.za/data-modeler-job-2493039.aspx'},
 {'Job Title': 'Data Analyst (Power BI)',
  'Company': 'Network Contracting Solutions',
  'Location': 'Johannesburg North (Johannesburg North), Gauteng',
  'Job Type': 'Contract Senior position',
  'Salary': '400.00 - R480.00 per hour',
  'Posting date': '25 Nov 2022',
  'Expiry date': '07 Dec 2022',
  'Link': 'https://www.careerjunction.co.za/data-analyst-power-bi-job-2491662.aspx'},
 {'Job Title': 'People Data Analyst at SA Taxi Development Finance',
  'Company': 'SA Taxi Development Finance',
  'Location': 'Midrand, Gauteng',
  'Job Type': 'Permanent Intermediate EE position',
  'Salary': 'Undisclosed',
  'Posting date': '25 Nov 

In [98]:
dfdf = pd.DataFrame(li)
dfdf

,Job Title,Company,Location,Job Type,Salary,Posting date,Expiry date,Link
0,Data Modeler,Lusapho Professional Services,"Johannesburg Region, Gauteng\n / Work From Home",Contract Specialist position,Undisclosed,26 Nov 2022,21 Dec 2022,/data-modeler-job-2493039.aspx
1,Data Analyst (Power BI),Network Contracting Solutions,"Johannesburg North (Johannesburg North), Gauteng",Contract Senior position,400.00 - R480.00 per hour,25 Nov 2022,07 Dec 2022,/data-analyst-power-bi-job-2491662.aspx
2,People Data Analyst at SA Taxi Development Fin...,SA Taxi Development Finance,"Midrand, Gauteng",Permanent Intermediate EE position,Undisclosed,25 Nov 2022,11 Dec 2022,/people-data-analyst-at-sa-taxi-development-fi...
3,Data Engineer Services,Tipp Focus,"Pretoria CBD, Gauteng",Contract Senior position,Undisclosed,25 Nov 2022,29 Dec 2022,/data-engineer-services-job-2494127.aspx
4,Junior Data Manager (12 Months Contract) Wits ...,AJ Personnel,"Johannesburg Region, Gauteng",Contract Junior position,Undisclosed,25 Nov 2022,25 Dec 2022,/junior-data-manager-12-months-contract-wits-n...
5,Senior Data Analyst,BigTalent,"Johannesburg, Gauteng\n / Work From Home",Permanent Senior position,"58,300 - R83,300 per month",25 Nov 2022,11 Dec 2022,/senior-data-analyst-job-2491573.aspx
6,Data Scientist,BigTalent,"Johannesburg, Gauteng\n / Work From Home",Permanent Junior position,"29,200 - R41,700 per month",25 Nov 2022,11 Dec 2022,/data-scientist-job-2491566.aspx
7,Senior Data Scientist,BigTalent,"Johannesburg Region, Gauteng\n / Work From Home",Permanent Senior position,"83,300 - R125,000 per month",25 Nov 2022,11 Dec 2022,/senior-data-scientist-job-2491571.aspx
8,TM1 Data Analytics Consultant at RecruiTech,RecruiTech,"Johannesburg, Gauteng\n / Work From Home",Permanent Intermediate position,"70,000 - R80,000 per month",25 Nov 2022,11 Dec 2022,/tm1-data-analytics-consultant-at-recruitech-j...
9,AWS Data Engineering Trainee - T_004,Jordan HR,"Pretoria, Gauteng",Contract Intermediate position,Undisclosed,24 Nov 2022,06 Dec 2022,/aws-data-engineering-trainee-t-004-job-249146...


In [ ]:
gen_url100 = "https://www.careerjunction.co.za/jobs/results?keywords=data&autosuggestEndpoint=%2fautosuggest&location=2747&category=&btnSubmit=+&page=2898"
resp100 = requests.get(gen_url100)
print(resp100, resp100.reason) 
print(resp100.status_code)

<Response [200]> OK
200


In [ ]:
pasta = BeautifulSoup(resp.content, 'html.parser')
jobs = pasta.find('div', class_="two-thirds")
j_cards = jobs.find_all('div', class_="job-result")
#j_cards.pop(5)
print(len(j_cards))
#j_cards

25


In [ ]:
li = []
for j_card in j_cards:
  j_today = date.today()
  a_job = {}
  a_job['Job Title'] = (j_card.find('h2').text.strip())
  a_job['Company'] = (j_card.find('h3').text.strip())
  a_job['Location'] = (j_card.find('li', class_="location").text.strip())
  a_job['Job Type'] = (j_card.find('li', class_="position").text.strip())
  a_job['Salary'] = (j_card.find('li', class_="salary").text.strip('R').strip())
  a_job['Posting date'] = (j_card.find('li', class_="updated-time").text.strip('Posted').strip())
  j_expiry = j_card.find('li', class_="expires").text.split()[2]
  if j_expiry != 'today':
    ini_exp = j_today + timedelta(days=int(j_expiry))
    a_job['Expiry date'] = ini_exp.strftime('%d %b %Y')
  else:
    a_job['Expiry date'] = j_today.strftime('%d %b %Y')
  #a_job['Job URL'] = (j_card.find('h2').text.strip())
  #a_job['j_title'] = (j_card.find('h2').text.strip())

  li.append(a_job)

#two-way step in accessing an embedded feature, find outer tag and assign it
#then use asigned_varaible.attrs['some_tag'] to locate the embedded feature
#eg -> var = j_card.find('h2') followed by feature = var.attrs['a'] 

#convert final list of dictionaries to a pandas dataframe then use to_csv after
#df = pd.DataFrame(jobs) followed by df.to_csv('file_name.csv')

len(li)
li[3]

{'Job Title': 'Data Scientist - Sandton (Hybrid role) - R1.1m per annum cost to company at e-Merge IT Recruitment',
 'Company': 'e-Merge IT Recruitment',
 'Location': 'Johannesburg, Gauteng',
 'Job Type': 'Permanent Senior position',
 'Salary': '91,700 per month',
 'Posting date': '23 Nov 2022',
 'Expiry date': '30 Nov 2022'}

In [ ]:
#while True:
#  try:
#    gen_url
#  except AttributeError:
#    break

#do the for loop shandis from here to start again

In [ ]:
#gen_url will change to url when get_url function is finally up and running
tsamaea = jobs.find('ul', {'id':'pagination'})
gen_url2 = tsamaea.find('a', class_="alt")
semate = gen_url[0]
print(gen_url2)
print(semate)


<a class="alt" href="https://www.careerjunction.co.za/jobs/results?keywords=data&amp;autosuggestEndpoint=%2fautosuggest&amp;location=2747&amp;category=&amp;btnSubmit=+&amp;page=1" rel="nofollow">&lt;</a>
h


In [ ]:
eew = [1,4]
rew = [1]
rew[-1]

1

In [ ]:
rea_today = datetime.now()#.strftime('%d %b %Y')
rea_today = rea_today + timedelta(days=1)
rea_today.strftime('%d %b %Y')

'25 Nov 2022'

## **Practice runs**

### *Prac 1*

In [ ]:
#pip install beautifulsoup4

In [ ]:
#from selenium import webdriver
#from bs4 import BeautifulSoup as bfs
#import pandas as pd

In [ ]:
#driver = webdriver.Chrome('/usr/lib/chromium-browser/chromedriver')

In [ ]:
#products = []
#prices = []
#ratings = []
#webdriver.get('''"<a href="https://www.flipkart.com/laptops/">https://www.flipkart.com/laptops/</a>~buyback-guarantee-on-laptops-/pr?sid=6bo%2Cb5g&amp;amp;amp;amp;amp;amp;amp;amp;amp;uniq"''')

In [ ]:
#pip install lxml


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#pip install html5lib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# extracting fifa 2022 group tables
#all_tables = pd.read_html("https://en.wikipedia.org/wiki/2022_FIFA_World_Cup")

In [ ]:
#all_tables[32]

,Poland,Match 39,Argentina
0,NaN,Report,NaN


In [ ]:
# the group tables begin at index 11 of all_tables, then all other groups are
# 7 indices from one another after the previous one. The following code outputs
# the index of every group
#import string
#alphabet = list(string.ascii_uppercase)

#for letter, ind in zip(alphabet, range(11, 67, 7)):
#  print(letter, ind)

A 11
B 18
C 25
D 32
E 39
F 46
G 53
H 60


In [ ]:
# organising the data
#dict_tables = dict()
#for letter, ind in zip(alphabet, range(11, 67, 7)):
#  df = all_tables[ind]
#  df.rename(columns={df.columns[1]: 'Team'}, inplace = True)
  #df.pop('Qualification')
#  dict_tables[f'Group {letter}'] = df

#dict_tables.keys()

dict_keys(['Group A', 'Group B', 'Group C', 'Group D', 'Group E', 'Group F', 'Group G', 'Group H'])

In [ ]:
#dict_tables['Group A'].Team

In [ ]:
 #for g_name, teams in dict_tables.items():
 #  print(g_name, dict_tables[g_name].Team)

In [ ]:
#import csv
#some_some = [((1,4),(2,4),(3,4),(4,4),(5,4),(6,4),(7,4),(8,4)),((1,4),(2,4),(3,4),(4,4),(5,4),(6,4),(7,4),(8,4))]
#with open('demofile.csv', 'w', newline='', encoding='utf-8') as xxxx:
  #for g_name, teams in dict_tables.items():
#  writer = csv.writer(xxxx)
#  writer.writerow(['Group A','Group B', 'Group C', 'Group D', 'Group E',\
#                     'Group F', 'Group G', 'Group H'])
#  writer.writerows(some_some)
    #xxxx.write(teams) #pops an error write() argument must be str, not DataFrame

In [ ]:
#fifa22_df = pd.DataFrame(dict_tables.items())
#fifa22_df

In [ ]:
#fifa22_df.to_csv('fifa2022.csv', index=False, encoding='utf-8')

### *Prac 2*

In [ ]:
# some more practice

def get_url(post, location):
  some_template = 'https://ie.indeed.com/jobs?q={}&l={}'
  url = some_template.format(post, location)
  return url

In [ ]:
some_url1 = get_url('Mechanic', 'Ireland')

In [ ]:
#exctrating the raw html
some_response1 = requests.get(some_url1)
print(some_response1)
print(some_response1.reason) #tells reason for request not being successful when applicable

In [ ]:
soup = BeautifulSoup(some_response1.text, 'html.parser')
some_cards = soup.find_all('div', 'job_seen_beacon')
len(some_cards)
#had to be the number of items displayed per page had response been 200 not 403
#just continue for now

In [ ]:
# getting data for the records
# card=some_cards[0], single record
def get_record(card):
  atag = some_cards.h2.a  #h2 heading, a-tag, use card to grab it
  job_title = card.find('span', 'title').text.strip()
  job_url = 'https://ie.indeed.com' + atag.get('href')
  job_company = card.find('span', 'companyName').text.strip()
  job_location = card.find('div', 'companyLocation').text.strip()
  #use.find(some_key).get('some_data') when desired variable is somewhat 
  #'embedded', without striping and texting it
  post_date = card.find('div', 'date').text
  some_today = datetime.today().strftime('%Y-%m-%d')
  try:
    job_salary = card.find('div','attribute_snippet').get('aria-label') #cant find it on current website though
  except AttributeError:
    job_salary = ''
  
  record = (job_title, job_company, job_location, post_date, some_today, job_salary, job_url)

  return record

In [ ]:
records = []
for card in some_cards:
  record = get_record(card)
  records.append(record)
records

In [ ]:
#next page
while True:
  try:
    url = 'https://ie.indeed.com' + soup.find('a', {'aria-label':"Next Page"}).get('href')
  except AttributeError:
    break
  
  some_response = requests.get(url)
  soup = BeautifulSoup(some_response1.text, 'html.parser')
  some_cards = soup.find_all('div', 'job_seen_beacon')

  for card in some_cards:
    record = get_record(card)
    records.append(record)

len(records)

In [ ]:
#consolidating

def some_main(some_job, some_location):
  records = []
  some_url = get_url(some_job, some_location)

  while True:
    some_response = requests.get(some_url)
    soup = BeautifulSoup(some_response1.text, 'html.parser')
    some_cards = soup.find_all('div', 'job_seen_beacon')

    for card in some_cards:
      record = get_record(card)
      records.append(record)

    try:
      url = 'https://ie.indeed.com' + soup.find('a', {'aria-label':"Next Page"}).get('href')
    except AttributeError:
      break
    #try-catch block at the end because we need to proceed to next page from the first

    #saving to csv
    with open('some_results.csv', 'x', newline='', encoding='utf-8') as some_f:
      writer = csv.writer(some_f)
      writer.writerow(['Job Title', 'Company', 'Location', "Posted On", 'Extracted On', "Salary", 'URL'])
      writer.writerows(records)


In [ ]:
some_main('Driver', 'Ireland')

### Prac 3

In [ ]:
resp1 = requests.get('https://www.irishjobs.ie/ShowResults.aspx?Keywords=Data&SortBy=Relevance&Page=2')

In [ ]:
resp1

### Prac 4

In [ ]:
def get_url(job):
  gen_url = "https://www.careerjunction.co.za/jobs/results?Keywords={}&lr=4&SortBy=Relevance"
  search_url = gen_url.format(job)
  return search_url

In [ ]:
resp3 = requests.get(get_url('data'))

In [ ]:
resp3

<Response [200]>

In [ ]:
soup = BeautifulSoup(resp3.text, 'lxml') #resp3.content to be tested
the_card = soup.find('div', "two-thirds")
print(len(the_card))
#the_card

61


In [ ]:
soup2 = BeautifulSoup(resp3.content, 'html.parser') #resp3.content to be tested
the_card2 = soup2.find('div', "two-thirds")

b_card = soup2.find_all('div', class_="module-content")
print(len(a_card))
#print(a_card)
l2 = []
for i in b_card:
  #MAMBOJUMBE = i.find('li', {'class':"salary"}).text
  #l2.append(MAMBOJUMBE)
  print(i)

len(l2)

In [ ]:
a_card = the_card.find_all('div', class_="module-content")
print(len(a_card))
print(a_card)
l1 = []
for i in a_card:
  l1.append(the_card.find('li', class_="position").text)

len(l1)

In [ ]:
records = dict()
j_location, j_type, j_salary, j_something =[],[],[],[]
#for card in j_card:
  
j_location = the_card.find_all('li', {'class':"location"})#.test.strip()
j_type = the_card.find_all('li', class_="position")#.text
j_salary = the_card.find_all('li', {'class':"salary"})#.text
j_something = the_card.find_all('h3')#.text.strip()
  #record = (j_location, j_type, j_salary, j_something)

for s_index, s_item in enumerate(j_salary):
  j_salary[s_index] = s_item.text.strip('R').strip()

for s_index, s_item in enumerate(j_type):
  j_type[s_index] = s_item.text.strip()

records['Location'] = j_location
records['Type'] = j_type
records['Salary'] = j_salary
records['Company'] = j_something

#print(len(records['Salary']), '\n', records['Salary'])
#print(len(records['Type']), '\n', records['Type'])
print(len(records['Company']))

25


In [ ]:
records = []
resp4 = requests.get('https://www.careerjunction.co.za/jobs/results?keywords=data')
soup = BeautifulSoup(resp4.text, 'html.parser')
the_card = soup.find_all('div', "two-thirds")

for i in j_card:   
  j_title = j_card.find('h2').text.strip()  #use i.find in normal circumstances
  j_location = j_card.find('li', "location").text.strip()
  j_type = j_card.find('li', "position").text
  j_salary = j_card.find('li', "salary").text
  j_company = j_card.find('h3').text.strip()
    
  record = (j_location, j_type, j_salary, j_something, j_title)
  records.append(record)

len(records)

In [ ]:
l1 = [1,2,3]
l2 = [76,73]
l1 += l2
l1